In [54]:
# ------------------------------------------------------------------------------------------------------------------------ #
# ---------------------------------------- 0.import library ---------------------------------------- # 
# ------------------------------------------------------------------------------------------------------------------------ #

# module : 설치가 되어 있지 않으면 설치 필요
import sys
import os
import xlrd
import pandas as pd
import numpy as np
import statistics
import math
import datetime
import time
import re # regural expression
import functools
import random
import gc 
import pickle
import warnings
import scipy
import sympy as sym
import datetime 

#
from sys import stdout
from functools import reduce 
from io import StringIO
from pandas.api.types import CategoricalDtype 
from itertools import groupby
from itertools import product
from itertools import chain
from itertools import cycle, islice
from collections import defaultdict
from collections import Counter
from scipy import stats 
from tqdm import tqdm

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns 
from matplotlib.ticker import MultipleLocator, IndexLocator, FuncFormatter
from matplotlib.dates import MonthLocator, DateFormatter
from matplotlib import gridspec



# check module version
print("- python version :", sys.version) # 파이썬 버전
print("- pandas version :", pd.__version__) # pandas 버전


# set options
pd.set_option('display.max_row', 500) 
pd.set_option('display.max_columns', 120) 

# 
print("- dir path :", os.getcwd())

# ignore warnings
warnings.filterwarnings("ignore")
# ------------------------------------------------------------------------------------------------------------------------ #

- python version : 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
- pandas version : 1.4.4
- dir path : \\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\pre\sampledata


In [18]:
# 2023년 장입량 데이터 : 2023년 월단위로 별도로 추가하여 갱신 필요
# xls 파일로 작업된 내용은 원본을 토대로 기존 항목에 맞춰 동일하게 항목명을 통일시킨 csv파일 활용함
# os.chdir(r"D:\국책과제_2차년도\DAT\rawdata\2차년도\charge장입량_year23_csv\mes")
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\rawdata\2차년도\charge장입량_year23_csv\mes")
chr_folderlist = os.listdir()

chr_file_ls = []
for chr_file in chr_folderlist : 

    print(chr_file)
    # import file
    chr_file_dt = pd.read_csv(chr_file, encoding = "cp949")
    chr_file_ls.append(chr_file_dt)
;

# 
input_ton_charge_2023 = pd.concat(chr_file_ls, axis = 0)
input_ton_charge_2023.sort_values("heat", inplace = True)

# 
input_ton_charge_2023 = input_ton_charge_2023[~pd.isna(input_ton_charge_2023["heat"])]
input_ton_charge_2023["heat"] = ["E" + str(int(x)) for x in input_ton_charge_2023["heat"]]
input_ton_charge_2023.rename(columns = {"heat" : "HEAT_NO"}, inplace = True)

# 
reshape_input_ton_charge_2023 = input_ton_charge_2023[["HEAT_NO", "1cha_ton", "2cha_ton", "3cha_ton"]].melt(id_vars = ["HEAT_NO"]) 
reshape_input_ton_charge_2023["ActualCHRG_PX3"] = [int(x[0]) for x in reshape_input_ton_charge_2023["variable"]]
reshape_input_ton_charge_2023.rename(columns = {"value" : "MES_ton"}, inplace = True)
reshape_input_ton_charge_2023["MES_ton"] = reshape_input_ton_charge_2023["MES_ton"].fillna(0)
reshape_input_ton_charge_2023["tot_mes_ton"] = reshape_input_ton_charge_2023.groupby(["HEAT_NO"])["MES_ton"].transform(lambda x: sum(x))
reshape_input_ton_charge_2023["장입횟수"] = reshape_input_ton_charge_2023.groupby(["HEAT_NO"])["MES_ton"].transform(lambda x : len(x) - sum(x ==0))

Apr_mes.csv
Aug_mes.csv
Dec_mes.csv
Feb_mes.csv
Jan_mes.csv
Jul_mes.csv
Jun_mes.csv
Mar_mes.csv
May_mes.csv
Nov_mes.csv
Oct_mes.csv
Sep_mes.csv


### 전기로 조업지표 
- 2023년 조업지표 sample로 진행

In [20]:
# os.chdir(r"D:\국책과제_2차년도\DAT\rawdata\2차년도")
os.chdir(r"\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\rawdata\2차년도")
steelinfo_dt = pd.read_csv("2023_전기로조업지표_sample.csv", header = 0, encoding = "cp949")

### SMART ARC
- 2023년 smart arc data sample로 진행
- 데이터에 따라 directory 변경 필요

In [21]:
# set data drectory
arc_file_fold = "\\\\10.216.153.164\\home\\begas\\27. AI활용 전기로 에너지 효율 최적화 기술개발\\DAT\\pre\\sampledata"
os.chdir(arc_file_fold)
print(os.getcwd()) 

\\10.216.153.164\home\begas\27. AI활용 전기로 에너지 효율 최적화 기술개발\DAT\pre\sampledata


In [22]:
filelists = os.listdir()
filelists

['2381460.csv',
 '2381810.csv',
 '2382010.csv',
 '2382070.csv',
 '2382140.csv',
 '2382210.csv',
 '2395640.csv',
 '2395820.csv',
 '2395880.csv',
 '2395950.csv',
 '2396000.csv',
 '2396130.csv']

In [23]:
# 필요 항목 정의 
column_arc_key = ["HEAT_NO", "ActualCHRG_PX3"]

In [24]:
target_name = "SMART ARC SAMPLE"

file_ls = []
for fname in filelists : 

    # get HEAT NO
    HEAT_NO = re.sub(".csv", "", fname)

    print("- dir : ", target_name, " | ", HEAT_NO)

    # import file
    temp_dt = pd.read_csv(fname)        


    # add HEAT NO
    temp_dt["HEAT_NO"] = "E" + str(HEAT_NO)
    temp_dt.rename(columns = {"Timestamp" : "Time"}, inplace = True)  

    # timestamp format 수정 : 일부 데이터의 format이 일관되지 않음
    # 동일한 형태로 통일 
    temp_dt["tm_tag"] = [str.split(x, " ")[1] for x in temp_dt["Time"]]

    # 
    if any(temp_dt["tm_tag"].isin(["오전", "오후"])) :

        # timestamp string 재정리  
        temp_dt["Time"] = [" ".join([x.split(" ")[0], str(np.where(x.split(" ")[1] == "오전", "AM", "PM")), x.split(" ")[2]]) for x in temp_dt["Time"]]

        # format 맞추기 
        temp_dt["Time"] = [datetime.datetime.strptime(x, "%Y-%m-%d %p %I:%M:%S") for x in temp_dt["Time"]]
       
        # 2개 case가 한 column에 동시에 존재 시 사전 처리 후 재변환 해야함 
        temp_dt["Time"] = [pd.to_datetime(x, format = "mixed") for x in temp_dt["Time"]]

    ;

    # 
    temp_dt.drop(columns = "tm_tag", inplace = True)

    try:

        # convert type to datetime(1) : set datetime
        temp_dt["Time"] = [pd.to_datetime(x) for x in temp_dt["Time"]]
        temp_dt["Time"] = temp_dt["Time"].astype('datetime64[ns]')

        # data sort by time
        temp_dt.sort_values("Time", ascending = True, inplace = True)

        # index reset 
        temp_dt.reset_index(inplace = True)

        # ------- 추가 : 데이터 항목 이상 보정 : 항목명이 달라진 항목들 기존 항목과 통일 -------  # 
        uline_dual_cols1 = [x for x in temp_dt.columns if "__" in x]
        uline_dual_cols2 = [x for x in temp_dt.columns if "___" in x]
        uline_dual_cols = uline_dual_cols1 + uline_dual_cols2

        # 
        origin_col_ls1 = [re.sub("__", "_", x) for x in uline_dual_cols1]
        origin_col_ls2 = [re.sub("___", "_", x) for x in uline_dual_cols2]
        origin_col_ls = origin_col_ls1 + origin_col_ls2

        if len(uline_dual_cols) >= 1 :

            #
            for x, x_origin in zip(uline_dual_cols, origin_col_ls) :

                # 
                x_new = x_origin + "_" + "newcols"

                # 이름 다르게 수집되었던 항목이 없는 경우 별도 처리 
                if x_origin in temp_dt.columns.tolist() :

                    temp_dt[x_new] = np.where(pd.isna(temp_dt[x_origin]), temp_dt[x], temp_dt[x_origin])
                    temp_dt.drop(columns = x, inplace = True)
                    temp_dt.drop(columns = x_origin, inplace = True)
                    temp_dt.columns = [re.sub("_newcols", "", x) for x in temp_dt.columns]

                else :

                    temp_dt[x_new] = temp_dt[x]
                    temp_dt.drop(columns = x, inplace = True)
                    temp_dt.columns = [re.sub("_newcols", "", x) for x in temp_dt.columns]
                ;
            ;
        ;

        # save
        file_ls.append(temp_dt)

        del temp_dt
        gc.collect()

    except:

        print("Exception HEAT : ", HEAT_NO, "\n")
        del temp_dt
        gc.collect()

        pass 
    ;

;

- dir :  SMART ARC SAMPLE  |  2381460
- dir :  SMART ARC SAMPLE  |  2381810
- dir :  SMART ARC SAMPLE  |  2382010
- dir :  SMART ARC SAMPLE  |  2382070
- dir :  SMART ARC SAMPLE  |  2382140
- dir :  SMART ARC SAMPLE  |  2382210
- dir :  SMART ARC SAMPLE  |  2395640
- dir :  SMART ARC SAMPLE  |  2395820
- dir :  SMART ARC SAMPLE  |  2395880
- dir :  SMART ARC SAMPLE  |  2395950
- dir :  SMART ARC SAMPLE  |  2396000
- dir :  SMART ARC SAMPLE  |  2396130


''

In [25]:
# 최종 데이터 병합 
rawdata_arc_copy = pd.concat(file_ls, axis = 0)

In [26]:
# HEAT_NO 변경
rawdata_arc_copy["HEAT_NO"] = [x[:7] for x in rawdata_arc_copy["HEAT_NO"]]

In [27]:
# 
# 데이터 전처리 : 장입횟수 3회 데이터만 가져오기 
# HEAT 번호, 장입차수 정보만 SMART ARC에서 추출 
cnt_ActualCHRG_PX3_1 = rawdata_arc_copy[["HEAT_NO", "ActualCHRG_PX3"]].drop_duplicates()

# HEAT 별 장입횟수, 장입차수 최대값, 장입차수 최소값 계산하기 
ActualCHRG_PX3_count = cnt_ActualCHRG_PX3_1.groupby("HEAT_NO").agg(count_value = ("ActualCHRG_PX3", lambda x : len(x))).reset_index()
ActualCHRG_PX3_max = cnt_ActualCHRG_PX3_1.groupby("HEAT_NO").agg(max_value = ("ActualCHRG_PX3", lambda x : max(x))).reset_index()
ActualCHRG_PX3_min = cnt_ActualCHRG_PX3_1.groupby("HEAT_NO").agg(min_value = ("ActualCHRG_PX3", lambda x : min(x))).reset_index()

# HEAT별 데이터 병합 
agg_ActualCHRG_PX3_1 = reduce(lambda x, y : pd.merge(x, y, on = "HEAT_NO"), [ActualCHRG_PX3_count, ActualCHRG_PX3_max, ActualCHRG_PX3_min])

# 장입이 3번이루어지고 ActualCHRG_PX3 번호 최소값이 1, 최대값이 3인 HEAT만 추출하기
Chg3_filter = agg_ActualCHRG_PX3_1[agg_ActualCHRG_PX3_1["count_value"] == 3]
Chg3_filter = Chg3_filter[Chg3_filter["min_value"] == 1]
Chg3_filter = Chg3_filter[Chg3_filter["max_value"] == 3]

# 3회 장입 데이터만 선택 
charge_cnt_3_heat = pd.unique(Chg3_filter.HEAT_NO)
arc_2022_SNOV_9_filter = rawdata_arc_copy[rawdata_arc_copy["HEAT_NO"].isin(charge_cnt_3_heat)]
print(arc_2022_SNOV_9_filter.shape[0])

arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter.drop_duplicates()
print(arc_2022_SNOV_9_filter.shape[0])

9931
9931


In [28]:
# 데이터 전처리 : 시작 종료지점 설정 
# charge 3 종료시점의 경우 데이터 마지막 시점까지 고려  
cols_1 = arc_2022_SNOV_9_filter.columns.tolist()
cols_1 = list(set(cols_1) - set(column_arc_key))

# 시작 종료지점 설정 : 승온기는 우선 포함됨 
def fn_set_start_end_point_prc(data) :

#     print("HEAT_NO : ", data["HEAT_NO"].values[0])

    # 신호값 변동지점 탐색 
    # 시작 : Charge_MWH가 0이 아니게 되는 시점 중 가장 처음 시점
    # 종료 : Charge_MWH 전 후 shift하여 비교 시 현재 시점으로부터 이전 시점과는 차이가 존재하나 이후 시점과는 차이가 없는 지점 중 가장 마지막 지점

    # 
    chr_n = data["ActualCHRG_PX3"].values[0]

    # time shift
    data["diff_ch_mwh"] = (data["Charge_MWH"] - data["Charge_MWH"].shift(1)).fillna(0)
    data["diff_ch_mwh_rev"] = (data["Charge_MWH"] - data["Charge_MWH"].shift(-1)).fillna(0)
    data["diff_PwrOn_rev"] = (data["TimeSincePwrOn"] - data["TimeSincePwrOn"].shift(-1)).fillna(0)

    # 시작, 종료시점 지정
    if data[data["Charge_MWH"] != 0].shape[0] < 1 :
        
        # Charge_MWH가 모두 0인 비정상 case가 존재하는 HEAT  : 제외 대상
        data["chr_start_tm"] = data["Time"].values[0]
        data["chr_end_tm"] = data["Time"].values[len(data["Time"]) - 1]
        data["total_time"] = -999
        data["time_search_tag"] = "abnormal"
        
    else : 
        
        # 시작시점
        chr_start_tm = min(data[data["Charge_MWH"] != 0]["Time"])
        
        # 종료시점 : TimesincePwerOn 값과 Charge_MWH를 동시에 확인
        if chr_n < 3 :

            # 
            chr_end_tm = max(data[(data["diff_ch_mwh"] != 0)&(data["diff_ch_mwh_rev"] == 0)]["Time"]) 
            data["chr_start_tm"] = chr_start_tm
            data["chr_end_tm"] = chr_end_tm
            data = data[data["Time"].between(chr_start_tm, chr_end_tm)]

        else :

            # TimesincePwerOn 값과 Charge_MWH를 동시에 확인
            timeseq = data[data["diff_PwrOn_rev"] > 0]["Time"]

            # TimeSincePwrOn만으로 결정이 안되는 경우 원래 로직으로 회귀 
            if len(timeseq) < 1 :

                chr_end_tm = max(data[(data["diff_ch_mwh"] != 0)&(data["diff_ch_mwh_rev"] == 0)]["Time"])

            else : 
                chr_end_tm = max(data[data["diff_PwrOn_rev"] > 0]["Time"])
            ; 

            # 
            data["chr_start_tm"] = chr_start_tm
            data["chr_end_tm"] = chr_end_tm
            data = data[data["Time"] >= chr_start_tm]
        ; 
    
        # 
        data["total_time"] = (chr_end_tm - chr_start_tm).total_seconds()/60
        data["time_search_tag"] = "normal"

    ; 

    # 
    return data
;

# 
arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter.groupby(["HEAT_NO", "ActualCHRG_PX3"]).apply(fn_set_start_end_point_prc)
arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter[cols_1 + ["diff_ch_mwh", "diff_ch_mwh_rev", "diff_PwrOn_rev", "chr_start_tm", "chr_end_tm", "total_time", "time_search_tag"]].reset_index()
arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter[column_arc_key + cols_1 + ["diff_ch_mwh", "diff_ch_mwh_rev", "diff_PwrOn_rev", "chr_start_tm", "chr_end_tm", "total_time", "time_search_tag"]]

# charge 별 조업시간 60분 이상 데이터 제외 
timeover_heat = pd.unique(arc_2022_SNOV_9_filter[arc_2022_SNOV_9_filter["total_time"] > 60]["HEAT_NO"])
# arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter[~arc_2022_SNOV_9_filter["HEAT_NO"].isin(timeover_heat)]

# 전력량 확인 안되는 데이터 제외
chrmwh_abnorm_heat = pd.unique(arc_2022_SNOV_9_filter[arc_2022_SNOV_9_filter["time_search_tag"] == "abnormal"]["HEAT_NO"])
# arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter[~arc_2022_SNOV_9_filter["HEAT_NO"].isin(chrmwh_abnorm_heat)]

# 파생변수 : 월 항목 생성
arc_2022_SNOV_9_filter["Month"] = arc_2022_SNOV_9_filter["Time"].dt.month


In [29]:
# 중복 수집된 HEAT 데이터 단일화 
arc_2022_SNOV_9_filter = arc_2022_SNOV_9_filter.drop_duplicates()

In [30]:
# HEAT 거르기 : 장입량 90 - 100톤 
in_90_100_heat = reshape_input_ton_charge_2023[reshape_input_ton_charge_2023["tot_mes_ton"].between(90, 100)]["HEAT_NO"]
in_90_100_heat = pd.unique(in_90_100_heat).tolist()

# 
arc_2023_analysis = arc_2022_SNOV_9_filter[arc_2022_SNOV_9_filter["HEAT_NO"].isin(in_90_100_heat)]

### 승온기 탐색
- 2023년 데이터 sample로 진행

In [32]:
# Post Consumtion Factor
arc_2023_analysis["Year"] = 2023

# PC FACTOR
arc_2023_analysis["PCFactor"] = arc_2023_analysis["NormCO2PctFlow"]/(arc_2023_analysis["NormCOPctFlow"] + arc_2023_analysis["NormCO2PctFlow"])

In [33]:
# charge 1, 2, 3 분리  
charge_1_dt = arc_2023_analysis[arc_2023_analysis["ActualCHRG_PX3"] == 1]
charge_2_dt = arc_2023_analysis[arc_2023_analysis["ActualCHRG_PX3"] == 2]
charge_3_dt = arc_2023_analysis[arc_2023_analysis["ActualCHRG_PX3"] == 3]

In [34]:
# 데이터 처리 : 측온온도 9999(error value)인 경우 -1로 수정 
charge_3_dt["PX3SteelTemp"] = np.where(charge_3_dt["PX3SteelTemp"] > 5000, -1, charge_3_dt["PX3SteelTemp"])

In [38]:
# len(condition_1_ls) 
finde_point_heatlist = pd.unique(charge_3_dt["HEAT_NO"]).tolist()

In [51]:
# 승온기 탐지를 위한 시작지점 탐색 
all_h_timedelta_ls = []
Errorheat_list = []

# 
for heat in tqdm(finde_point_heatlist) : 
    
    # 
    print("- heat : ", heat)

    # 
    data = charge_3_dt[charge_3_dt["HEAT_NO"] == heat]

    # 
    data["c1"] = np.where(data["TimeSincePwrOn"] <= 0.1, 1, 0) # 0이 되는 signal loss 존재 
    data["c2"] = np.where(data["PX3SteelTemp"] > 0, 1, 0)

    # Power On이 지속되기만 하면 진행 불가 
    if sum(data["c1"]) == 0 : 

        # 
        print("- Condition 1 ERROR HEAT : ", heat)
        CODE = "Condition 1 False"
        check_tag = "TimeSincePwrOn all upper crit value"
        info_dt = pd.DataFrame({"CODE" : CODE, "check_tag" : check_tag, "HEAT_NO" : heat}, index = [0])
        Errorheat_list.append(info_dt)

        continue 
    ;

    # 측온값이 전혀 없으면 진행 불가 
    if sum(data["c2"]) == 0 : 

        # 
        print("- Condition 1 ERROR HEAT : ", heat)
        CODE = "Condition 2 False"
        check_tag = "PX3SteelTemp all zero"
        info_dt = pd.DataFrame({"CODE" : CODE, "check_tag" : check_tag, "HEAT_NO" : heat}, index = [0])
        Errorheat_list.append(info_dt)

        continue 
    ;
    
    try : 
        
        # 
        data["diff_PwrOn"] = (data["TimeSincePwrOn"] - data["TimeSincePwrOn"].shift(1)).fillna(np.nan)
        data["diff_PwrOn_flag"] = np.where(data["diff_PwrOn"] < 0, 1, 0)
        data["int_num"] = np.cumsum(data["diff_PwrOn_flag"]) + 1

        # 측온온도값이 존재하는 구간 찾기 
        target_start_int = min(data[data["PX3SteelTemp"] > 0]["int_num"])

        # 구간 내 실제 Power On 시작지점 찾기 
        # >> TimeSincePwrOn이 0이 아닌 최소값을 가지는 지점 
        Temp_inc_st = min(data[(data["int_num"] == target_start_int)&(data["TimeSincePwrOn"] > 0)]["Time"])
        data["change_point"] = np.where(data["Time"] == Temp_inc_st, 1, 0)
        data["Inc_Temp_tag"] = np.where(data["Time"] > Temp_inc_st, "Int_Heating", "Int_Melting")

        # 
        all_h_timedelta_ls.append(data)

    except : 
        
        # 
        print("- Tracking ERROR HEAT : ", heat)
        CODE = "Tracking ERROR"
        check_tag = "Not found change points"
        info_dt = pd.DataFrame({"CODE" : CODE, "check_tag" : check_tag, "HEAT_NO" : heat}, index = [0])
        Errorheat_list.append(info_dt)

        continue

    ;

;



100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 128.22it/s]

- heat :  E238146
- heat :  E238181
- heat :  E238201
- heat :  E238207
- heat :  E238214
- Condition 1 ERROR HEAT :  E238214
- heat :  E238221
- heat :  E239564
- heat :  E239582
- heat :  E239588
- heat :  E239595
- heat :  E239600
- heat :  E239613


''

In [57]:
# result binding
find_timedelta_binding = pd.concat(all_h_timedelta_ls, axis = 0)
Errorheat_list_binding = pd.concat(Errorheat_list, axis = 0)

In [130]:
Errorheat_list_binding_heat = Errorheat_list_binding["HEAT_NO"]

### charge 3 용해기 / 승온기 집계 

In [59]:
# 용해기 승온기 구분 
find_timedelta_heatlist = pd.unique(find_timedelta_binding.HEAT_NO).tolist()

### [3차년도] 로직 수정 필요 : 온도 값 보정을 하는 경우와 안하는 경우의 tag 구분

In [90]:
# # 구간 별 측온온도 편차 확인 : 승온량
# # 구간 간 비교 시 승온온도가 데이터 상으로 하강하는 경우는 비정상 case : 데이터 Error로 추정
# # >> 이 경우는 측온 온도값을 직전 구간 온도로 보정 후 별도 인식 tag 부여  


# # new columns copy
# save_vcb_int_dt["Temp_vcb_new"] = save_vcb_int_dt["Temp_vcb"].copy()

# # 중간구간 측온온도가 -1인 경우 : 데이터 수집 error
# # 구간 승온량 확인
# save_vcb_int_dt["delta_T_vcb"] = (save_vcb_int_dt["Temp_vcb_new"] - save_vcb_int_dt["Temp_vcb_new"].shift(1)).fillna(0)

# # 보정여부 Tag : 승온량이 0 미만인 경우 check
# save_vcb_int_dt["delta_T_tag"] = np.where(any(save_vcb_int_dt["delta_T_vcb"] < 0), "value_correction", "normal_case")

# # 측온온도값 보정 : 데이터 Error로 인해 측온온도가 역전되는 현상이 발생 시 이 현상이 없어질 때까지 단계적으로 보정 
# # 온도 Error값 결측 변환 
# save_vcb_int_dt["Temp_vcb_new"] = np.where(save_vcb_int_dt["delta_T_vcb"] < 0, 
#                                            np.nan, save_vcb_int_dt["Temp_vcb"])

# # 결측데이터 보정
# save_vcb_int_dt["Temp_vcb_new"] = save_vcb_int_dt["Temp_vcb_new"].fillna(method = 'pad')

# # 구간 승온량 확인(re)
# save_vcb_int_dt["delta_T_vcb"] = (save_vcb_int_dt["Temp_vcb_new"] - save_vcb_int_dt["Temp_vcb_new"].shift(1)).fillna(0)


### [3차년도] 로직 수정 필요 : 해당 case인 경우 승온량 및 측온온도의 경우 보정값 대체가 아닌 원래 값으로 두고 사후 처리여부 결정 필요

In [91]:
# # 가장 마지막 구간 측온온도가 0이 되는 경우 : 비교대상이 없으므로 보정이 안될수 있음
# # 이 경우 구간승온량이 0 이하가 될 수 있음

# if save_vcb_int_dt["delta_T_vcb"].values[-1] < 0 : 

#     # 구간 측온값 보정(출강온도) : 직전 구간 출강온도로 보정 
#     save_vcb_int_dt.loc[save_vcb_int_dt["delta_T_vcb"] < 0, "Temp_vcb_new"] = save_vcb_int_dt["Temp_vcb_new"].values[-2] 
    
#     # 승온량 계산 
#     save_vcb_int_dt["delta_T_vcb"] = (save_vcb_int_dt["Temp_vcb_new"] - save_vcb_int_dt["Temp_vcb_new"].shift(1)).fillna(0)
# ;


# # 승온량 구간 별 matching 
# save_vcb_int_dt["delta_T_calc"] = (save_vcb_int_dt["delta_T_vcb"].shift(-1)).fillna(0)

####  전체 code 실행

In [93]:
total_save_melting_int_ls = []
total_save_heating_int_ls = []
total_save_hvcb_int_ls = []

#
for heat in tqdm(find_timedelta_heatlist) :

    # heat = "E233900"
    print("- HEAT NO : ", heat)
    
    # 
    data = find_timedelta_binding[find_timedelta_binding["HEAT_NO"] == heat]


    try :

        # 
        first_ch_point_tm = min(data[data["change_point"] == 1]["Time"])

        # 1. 용해기 : 첫 구분 tag에 해당되는 시간 이전 데이터 중 최대 누적 전력사용량을 가지는 구간까지 
        ch3_melting_dt = data[data["Time"] < first_ch_point_tm]
        ch3_melting_dt = ch3_melting_dt[ch3_melting_dt["Charge_MWH"] <= max(ch3_melting_dt["Charge_MWH"])]

        # 1-1. 용해기 : 용해시간, 용해 전력사용량, 노벽산소, 노전산소, 가탄재  
        ch3_melting_time = (max(ch3_melting_dt["Time"]) - min(ch3_melting_dt["Time"])).total_seconds()/60
        ch3_melting_mwh = max(ch3_melting_dt["Charge_MWH"])
        ch3_melting_burnero2 = max(ch3_melting_dt["PX3BurnerO2"]) - min(ch3_melting_dt["PX3BurnerO2"])
        ch3_melting_lanceo2 = max(ch3_melting_dt["PX3LanceO2"]) - min(ch3_melting_dt["PX3LanceO2"])
        ch3_melting_injcarb = max(ch3_melting_dt["PX3InjecCarbon"]) - min(ch3_melting_dt["PX3InjecCarbon"])
        
        # 
        measure_last_melt_mwh = max(ch3_melting_dt["MWH_PX3"])

        # 1-2 용해기의 VCB OFF 횟수 
        ch3_melting_dt["diff_PwrOn"] = (ch3_melting_dt["TimeSincePwrOn"] - ch3_melting_dt["TimeSincePwrOn"].shift(1)).fillna(0)
        ch3_melting_dt["cat_diff_PwrOn"] = np.where(ch3_melting_dt["diff_PwrOn"] < 0, 1, 0)
        cnt_vcb_off_melt = ch3_melting_dt[ch3_melting_dt["cat_diff_PwrOn"] == 1].shape[0]

        # 구간 시작/종료 시점 : tracking 목적 
        melting_start_tm = min(ch3_melting_dt["Time"])
        melting_end_tm = max(ch3_melting_dt["Time"])

        # 
        save_melting_int_dt = pd.DataFrame({"HEAT_NO" : heat, 
                                            "measure_last_melt_mwh" : measure_last_melt_mwh, 
                                            "heating_time_vcb" : ch3_melting_time,
                                            "heating_mwh_vcb" : ch3_melting_mwh, 
                                            "melting_start_tm" : melting_start_tm,
                                            "melting_end_tm" : melting_end_tm,
                                            "heating_burnero2_vcb" : ch3_melting_burnero2, 
                                            "heating_lanceo2_vcb" : ch3_melting_lanceo2, 
                                            "heating_injcarb_vcb" : ch3_melting_injcarb,
                                            "cnt_vcb_off_melt" : cnt_vcb_off_melt}, index = [0])

        
        # 2. 승온기 : 첫 구분 tag에 해당되는 시간 이후 데이터 중 TimesincePwrOn이 처음으로 인가되는 시점부터 종료시점까지 
        ch3_heating_dt = data[data["Time"] >= first_ch_point_tm]
        heating_start_tm = min(ch3_heating_dt[ch3_heating_dt["TimeSincePwrOn"] > 0]["Time"])
        ch3_heating_dt = ch3_heating_dt[ch3_heating_dt["Time"] >= heating_start_tm]
        

        # 2-1. 승온기(전체): 용해시간, 용해 전력사용량, 노벽산소, 노전산소, 가탄재  
        ch3_heating_time = (max(ch3_heating_dt["Time"]) - min(ch3_heating_dt["Time"])).total_seconds()/60
        ch3_heating_mwh = max(ch3_heating_dt["Charge_MWH"]) - ch3_melting_mwh
        ch3_heating_burnero2 = max(ch3_heating_dt["PX3BurnerO2"]) - min(ch3_heating_dt["PX3BurnerO2"])
        ch3_heating_lanceo2 = max(ch3_heating_dt["PX3LanceO2"]) - min(ch3_heating_dt["PX3LanceO2"])
        ch3_heating_injcarb = max(ch3_heating_dt["PX3InjecCarbon"]) - min(ch3_heating_dt["PX3InjecCarbon"])

        # 구간 시작/종료 시점 : tracking 목적 
        heating_start_tm = min(ch3_heating_dt["Time"])
        heating_end_tm = max(ch3_heating_dt["Time"])

        # 
        save_heating_int_dt = pd.DataFrame({"HEAT_NO" : heat, 
                                            "heating_time_vcb" : ch3_heating_time,
                                            "heating_mwh_vcb" : ch3_heating_mwh,
                                            "heating_start_tm" : heating_start_tm,  
                                            "heating_end_tm" : heating_end_tm,
                                            "heating_burnero2_vcb" : ch3_heating_burnero2, 
                                            "heating_lanceo2_vcb" : ch3_heating_lanceo2, 
                                            "heating_injcarb_vcb" : ch3_heating_injcarb}, index = [0])

        
        # 2-2. 승온기(실제 승온작업): 용해시간, 용해 전력사용량, 노벽산소, 노전산소, 가탄재  
        # VCB ON / OFF 구분하기 
        ch3_heating_dt["diff_PwrOn"] = (ch3_heating_dt["TimeSincePwrOn"] - ch3_heating_dt["TimeSincePwrOn"].shift(1)).fillna(0)
        ch3_heating_dt["cat_change_vcb"] = np.where((ch3_heating_dt["diff_PwrOn"] < 0), 1, 0)

        # change point search
        check_int_count = ch3_heating_dt[ch3_heating_dt["cat_change_vcb"] == 1].shape[0]

        # data copy
        vcb_int_dt = ch3_heating_dt.copy()

        # 구간 id 확인 
        int_num_ls = pd.unique(ch3_heating_dt["int_num"]).tolist()

        # 
        if check_int_count == 0 :

            # 이 경우 승온기 VCB Off 발생안함
            cnt_vcb_off = 0
            cnt_heating_interval = 1
            id_interval = 1
            heating_time_vcb = ch3_heating_time
            heating_mwh_vcb = ch3_heating_mwh
            heating_burnero2_vcb = ch3_heating_burnero2
            heating_lanceo2_vcb = ch3_heating_lanceo2
            heating_injcarb_vcb = ch3_heating_injcarb

            # 구간 용강 측온 온도 
            Temp_vcb = ch3_heating_dt["PX3SteelTemp"].values[0]

            # 승온 온도 
            delta_T_vcb = ch3_heating_dt["PX3SteelTemp"].values[-1] - ch3_heating_dt["PX3SteelTemp"].values[0]

            # 구간 시작/종료 시점 : tracking 목적 
            int_start_tm = min(ch3_heating_dt["Time"])
            int_end_tm = max(ch3_heating_dt["Time"])

            # save 
            save_vcb_int_dt = pd.DataFrame({"HEAT_NO" : heat, 
                                            "cnt_vcb_off" : cnt_vcb_off, 
                                            "cnt_heating_interval" : cnt_heating_interval, 
                                            "int_start_tm" : int_start_tm, 
                                            "int_end_tm" : int_end_tm, 
                                            "heating_time_vcb" : heating_time_vcb,
                                            "heating_mwh_vcb" : heating_mwh_vcb, 
                                            "heating_burnero2_vcb" : heating_burnero2_vcb, 
                                            "heating_lanceo2_vcb" : heating_lanceo2_vcb, 
                                            "heating_injcarb_vcb" : heating_injcarb_vcb,
                                            "id_interval" : id_interval,
                                            "Temp_vcb" : Temp_vcb,
                                            "delta_T_vcb" : delta_T_vcb}, index = [0])


        else : 

            # 
            save_vcb_int_sub = []

            # 이 경우 승온기 VCB Off 발생 횟수는 int_num 길이에 따름 : 
            cnt_vcb_off = len(int_num_ls) - 1
            cnt_heating_interval = len(int_num_ls)

            #
            for i, inm in enumerate(int_num_ls) :

                # 
                id_interval = i + 1

                # 
                vcb_int_sub = vcb_int_dt[vcb_int_dt["int_num"] == inm]
                int_pwr_on = vcb_int_sub[vcb_int_sub["TimeSincePwrOn"] > 0].shape[0]

                # 
                if int_pwr_on == 0:

                    # TimeSincePwrOn refresh 이후 전원이 마지막까지 인가되지 않는 구간 : 일반적으로는 종료구간 
                    # 마지막 측온온도 정도만 유효함
                    int_tag = "All_PwrOff_interval"

                    # 지표 계산 
                    heating_time_vcb = (max(vcb_int_sub["Time"]) - min(vcb_int_sub["Time"])).total_seconds()/60
                    int_mwh_max = max(vcb_int_sub["Charge_MWH"])
                    
                    heating_burnero2_vcb = max(vcb_int_sub["PX3BurnerO2"]) - min(vcb_int_sub["PX3BurnerO2"])
                    heating_lanceo2_vcb = max(vcb_int_sub["PX3LanceO2"]) - min(vcb_int_sub["PX3LanceO2"])
                    heating_injcarb_vcb = max(vcb_int_sub["PX3InjecCarbon"]) - min(vcb_int_sub["PX3InjecCarbon"])

                    # 마지막 측온온도
                    Temp_vcb = max(vcb_int_sub["PX3SteelTemp"])

                    # 구간 시작/종료 시점 : tracking 목적 
                    int_start_tm = min(vcb_int_sub["Time"])
                    int_end_tm = max(vcb_int_sub["Time"])

                    # 
                    agg_vcb_int_sub = pd.DataFrame({"HEAT_NO" : heat, 
                                                    "cnt_vcb_off" : cnt_vcb_off, 
                                                    "cnt_heating_interval" : cnt_heating_interval, 
                                                    "int_start_tm" : int_start_tm,
                                                    "int_end_tm" : int_end_tm,
                                                    "heating_time_vcb" : heating_time_vcb,
                                                    "int_mwh_max" : int_mwh_max,
                                                    "heating_burnero2_vcb" : heating_burnero2_vcb, 
                                                    "heating_lanceo2_vcb" : heating_lanceo2_vcb, 
                                                    "heating_injcarb_vcb" : heating_injcarb_vcb,
                                                    "int_num" : inm,
                                                    "id_interval" : id_interval,
                                                    "int_tag" : int_tag,
                                                    "Temp_vcb" : Temp_vcb}, index = [0])

                    # 
                    save_vcb_int_sub.append(agg_vcb_int_sub)


                else : 

                    int_tag = "Normal_PwrOn_interval"

                    # 유효한 전력투입 구간만 확인 
                    vcb_int_sub = vcb_int_sub[vcb_int_sub["TimeSincePwrOn"] > 0]

                    # 지표 계산 
                    heating_time_vcb = (max(vcb_int_sub["Time"]) - min(vcb_int_sub["Time"])).total_seconds()/60
                    int_mwh_max = max(vcb_int_sub["Charge_MWH"])
                    heating_burnero2_vcb = max(vcb_int_sub["PX3BurnerO2"]) - min(vcb_int_sub["PX3BurnerO2"])
                    heating_lanceo2_vcb = max(vcb_int_sub["PX3LanceO2"]) - min(vcb_int_sub["PX3LanceO2"])
                    heating_injcarb_vcb = max(vcb_int_sub["PX3InjecCarbon"]) - min(vcb_int_sub["PX3InjecCarbon"])

                    #구간 용강 측온 온도
                    # 구간 내 용강 측온 온도 편차 확인
                    Temp_in_delta = max(vcb_int_sub["PX3SteelTemp"]) - min(vcb_int_sub["PX3SteelTemp"])

                    # 
                    if Temp_in_delta != 0 :

                        # 이 경우는 측온되었던 데이터가 흔들린 경우
                        # 첫 구간인 경우 최초 측온값, 아니라면 나중 측온값으로 측온 온도를 결정 
                        if id_interval == 1:

                            Temp_vcb = vcb_int_sub["PX3SteelTemp"].values[0]

                        else :

                            Temp_vcb = vcb_int_sub["PX3SteelTemp"].values[-1]
                        ;

                    else :  

                        # 정상 case : 측온 온도값이 변동이 없음
                        Temp_vcb = vcb_int_sub["PX3SteelTemp"].values[0]

                    ;


                    # 구간 시작/종료 시점 : tracking 목적 
                    int_start_tm = min(vcb_int_sub["Time"])
                    int_end_tm = max(vcb_int_sub["Time"])

                    # 
                    agg_vcb_int_sub = pd.DataFrame({"HEAT_NO" : heat, 
                                                    "cnt_vcb_off" : cnt_vcb_off, 
                                                    "cnt_heating_interval" : cnt_heating_interval, 
                                                    "int_start_tm" : int_start_tm,
                                                    "int_end_tm" : int_end_tm,
                                                    "heating_time_vcb" : heating_time_vcb,
                                                    "int_mwh_max" : int_mwh_max,
                                                    "heating_burnero2_vcb" : heating_burnero2_vcb, 
                                                    "heating_lanceo2_vcb" : heating_lanceo2_vcb, 
                                                    "heating_injcarb_vcb" : heating_injcarb_vcb,
                                                    "int_num" : inm,
                                                    "id_interval" : id_interval,
                                                    "int_tag" : int_tag,
                                                    "Temp_vcb" : Temp_vcb}, index = [0])

                    # 
                    save_vcb_int_sub.append(agg_vcb_int_sub)

                ;

            ;

            # 
            save_vcb_int_dt = pd.concat(save_vcb_int_sub, axis = 0)

        ;


        # add index 
        save_vcb_int_dt["index"] = range(0, save_vcb_int_dt.shape[0])
        
        
        # 승온구간 별 전력사용량 계산 
        save_vcb_int_dt["heating_mwh_vcb"] = save_vcb_int_dt["int_mwh_max"] - save_vcb_int_dt["int_mwh_max"].shift(1)
        
        # 첫 승온구간 전력 : 용해기까지 사용했던 전력과 비교하여 계산 
        first_int_heating_mwh = save_vcb_int_dt[save_vcb_int_dt["id_interval"] == min(save_vcb_int_dt["id_interval"])]["int_mwh_max"].values[0]
        save_vcb_int_dt["heating_mwh_vcb"] = np.where(pd.isna(save_vcb_int_dt["heating_mwh_vcb"]), 
                                                      first_int_heating_mwh - ch3_melting_mwh, 
                                                      save_vcb_int_dt["heating_mwh_vcb"])

        # 
        # 구간 별 측온온도 편차 확인 : 승온량
        # 승온온도가 데이터 상으로 하강하는 경우는 비정상 case : 데이터 Error로 추정

        # >> 이 경우는 측온 온도값을 직전 구간 온도로 보정 후 별도 인식 tag 부여  
        save_vcb_int_dt["Temp_vcb_new"] = save_vcb_int_dt["Temp_vcb"].copy()

        # 측온온도가 -1인 경우 
        # 데이터 수집 error인 부분으로 직전 구간 온도값으로 보정
        save_vcb_int_dt["delta_T_vcb"] = (save_vcb_int_dt["Temp_vcb_new"] - save_vcb_int_dt["Temp_vcb_new"].shift(1)).fillna(0)
        save_vcb_int_dt["delta_T_tag"] = np.where(any(save_vcb_int_dt["delta_T_vcb"] < 0), "value_correction", "normal_case")

        # 측온온도값 보정 
        # 측온온도가 역전되는 현상이 없어질 때까지 단계적으로 보정 
        save_vcb_int_dt["Temp_vcb_new"] = np.where(save_vcb_int_dt["delta_T_vcb"] < 0, 
                                                   np.nan, save_vcb_int_dt["Temp_vcb"])
        save_vcb_int_dt["Temp_vcb_new"] = save_vcb_int_dt["Temp_vcb_new"].fillna(method = 'pad')
        save_vcb_int_dt["delta_T_vcb"] = (save_vcb_int_dt["Temp_vcb_new"] - save_vcb_int_dt["Temp_vcb_new"].shift(1)).fillna(0)

        # 가장 마지막 구간 측온온도가 0이 되는 경우 : 비교대상이 없으므로 보정이 안될수 있음
        if save_vcb_int_dt["delta_T_vcb"].values[-1] < 0 : 

            # 
            save_vcb_int_dt.loc[save_vcb_int_dt["delta_T_vcb"] < 0, "Temp_vcb_new"] = save_vcb_int_dt["Temp_vcb_new"].values[-2] 
            save_vcb_int_dt["delta_T_vcb"] = (save_vcb_int_dt["Temp_vcb_new"] - save_vcb_int_dt["Temp_vcb_new"].shift(1)).fillna(0)
        ;


        # 승온량 구간 별 matching 조정
        save_vcb_int_dt["delta_T_calc"] = (save_vcb_int_dt["delta_T_vcb"].shift(-1)).fillna(0)


        # save all 
        total_save_melting_int_ls.append(save_melting_int_dt)
        total_save_heating_int_ls.append(save_heating_int_dt)
        total_save_hvcb_int_ls.append(save_vcb_int_dt)


    except : 

        print("aggregate ERROR : ", heat)

    ;

;


 73%|████████████████████████████████████████████████████████████▎                      | 8/11 [00:00<00:00, 39.39it/s]

- HEAT NO :  E238146
- HEAT NO :  E238181
- HEAT NO :  E238201
- HEAT NO :  E238207
- HEAT NO :  E238221
- HEAT NO :  E239564
- HEAT NO :  E239582
- HEAT NO :  E239588


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 40.47it/s]

- HEAT NO :  E239595
- HEAT NO :  E239600
- HEAT NO :  E239613


''

In [96]:
# 모든 HEAT 데이터 집게 : DataFrame
total_save_melting_int_dt = pd.concat(total_save_melting_int_ls, axis = 0)
total_save_heating_int_dt = pd.concat(total_save_heating_int_ls, axis = 0)
total_save_hvcb_int_dt = pd.concat(total_save_hvcb_int_ls, axis = 0)

In [97]:
# 집계 오류 발생한 heat check
aggregate_error_heat = list(set(find_timedelta_heatlist) - set(total_save_melting_int_dt["HEAT_NO"]))

In [99]:
# 조업지표에서 집계 결과를 확인하지 못하는 경우에 대한 Tag
# resultcheck_dt = steelinfo_dt[["HEAT_NO"]]
# resultcheck_dt["비고"] = np.where(resultcheck_dt["HEAT_NO"].isin(noninclude_condition_heat), "전처리 조건 제외 : 장입 3회 or 장입량 90-100",
#                                np.where(resultcheck_dt["HEAT_NO"].isin(Errorheat_list_binding_heat), "승온기 탐색불가",
#                                        np.where(resultcheck_dt["HEAT_NO"].isin(aggregate_error_heat), "승온기 집계오류", np.nan)))

### charge 전력사용량 계산


In [132]:
Total_heat_MWH_PX3 = arc_2023_analysis[["HEAT_NO", "MWH_PX3"]].groupby(["HEAT_NO"])["MWH_PX3"].agg(["max"]).reset_index()
Total_heat_MWH_PX3

,HEAT_NO,max
0,E238146,34.68750
1,E238181,34.59375
2,E238201,34.40625
3,E238207,35.81250
4,E238214,33.31250
5,E238221,39.81250
6,E239564,35.81250
7,E239582,34.90625
8,E239588,35.59375
9,E239595,34.90625


In [103]:
# charge 1 
charge_1_max_mwh = charge_1_dt.groupby(["HEAT_NO"]).agg(ch1_max = ("MWH_PX3", lambda x : max(x))).reset_index()

# charge 2
charge_2_max_mwh = charge_2_dt.groupby(["HEAT_NO"]).agg(ch2_max = ("MWH_PX3", lambda x : max(x))).reset_index()
charge_2_max_mwh

,HEAT_NO,ch2_max
0,E238146,23.000000
1,E238181,21.500000
2,E238201,22.296875
3,E238207,23.203125
4,E238214,22.796875
5,E238221,22.593750
6,E239564,23.796875
7,E239582,23.000000
8,E239588,23.406250
9,E239595,23.906250


In [104]:
# 데이터 병합
MWH_calculate = reduce(lambda x, y : pd.merge(x, y, on = "HEAT_NO", how = "left"),
                       [charge_1_max_mwh, 
                        charge_2_max_mwh, 
                        total_save_melting_int_dt[["HEAT_NO", "measure_last_melt_mwh"]]])

# charge 1, 2 전력량 계산
MWH_calculate["charge_1_mwh"] = MWH_calculate["ch1_max"] 
MWH_calculate["charge_2_mwh"] = MWH_calculate["ch2_max"] - MWH_calculate["ch1_max"]

# charge 3 용락 전력량 계산 
MWH_calculate["charge_3_mwh"] = MWH_calculate["measure_last_melt_mwh"] - MWH_calculate["ch2_max"]
MWH_calculate

,HEAT_NO,ch1_max,ch2_max,measure_last_melt_mwh,charge_1_mwh,charge_2_mwh,charge_3_mwh
0,E238146,15.203125,23.000000,29.593750,15.203125,7.796875,6.593750
1,E238181,15.000000,21.500000,30.703125,15.000000,6.500000,9.203125
2,E238201,15.398438,22.296875,29.406250,15.398438,6.898438,7.109375
3,E238207,15.500000,23.203125,29.406250,15.500000,7.703125,6.203125
4,E238214,14.796875,22.796875,NaN,14.796875,8.000000,NaN
5,E238221,14.703125,22.593750,34.093750,14.703125,7.890625,11.500000
6,E239564,16.000000,23.796875,29.796875,16.000000,7.796875,6.000000
7,E239582,15.203125,23.000000,34.812500,15.203125,7.796875,11.812500
8,E239588,16.000000,23.406250,28.593750,16.000000,7.406250,5.187500
9,E239595,15.703125,23.906250,31.203125,15.703125,8.203125,7.296875


### 승온 별 측온온도 / 승온전력량

In [105]:
# 측온횟수 별 측온값 / 측온횟수 별 전력사용량
# 데이터 분리 
temp_measure_dt = total_save_hvcb_int_dt[["HEAT_NO", "int_tag", "cnt_vcb_off", "id_interval", 
                                          "heating_mwh_vcb", "Temp_vcb_new", "delta_T_calc"]]

In [112]:
temp_measure_dt[temp_measure_dt["HEAT_NO"] == "E238181"]

,HEAT_NO,int_tag,cnt_vcb_off,id_interval,heating_mwh_vcb,Temp_vcb_new,delta_T_calc,inc_count_0,inc_count,inc_count_max,id_str0,id_str,measure_temp_2
0,E238181,Normal_PwrOn_interval,1,1,3.898438,1540.0,91.0,1,1,1,measure_1,measure_2,1631.0
0,E238181,All_PwrOff_interval,1,2,0.000000,1631.0,0.0,0,1,1,measure_2,measure_3,1631.0


In [106]:
# 정상 승온 구간이면서 전력이 인가된 경우만 check
temp_measure_dt["inc_count_0"] = np.where((temp_measure_dt["int_tag"] == "Normal_PwrOn_interval")&(temp_measure_dt["heating_mwh_vcb"] != 0), 1, 0)


In [107]:
# 승온 횟수 확인
temp_measure_dt["inc_count"] = temp_measure_dt.groupby(["HEAT_NO"])["inc_count_0"].transform(lambda x : np.cumsum(x))
temp_measure_dt["inc_count_max"] = temp_measure_dt.groupby(["HEAT_NO"])["inc_count"].transform(lambda x : max(x))

In [108]:
# 구간 id tag
temp_measure_dt["id_str0"] = ["measure" + "_" + str(x) for x in temp_measure_dt["id_interval"]]
temp_measure_dt["id_str"] = ["measure" + "_" + str(x + 1) for x in temp_measure_dt["id_interval"]]

In [113]:
# 구간 별 승온 후 온도 확인
temp_measure_dt["measure_temp_2"] = temp_measure_dt["Temp_vcb_new"] + temp_measure_dt["delta_T_calc"]
temp_measure_check_n = temp_measure_dt[temp_measure_dt["int_tag"] == "Normal_PwrOn_interval"]

In [115]:
# 첫 승온구간 분리 
temp_measure_check_int1 = temp_measure_check_n[temp_measure_check_n["id_interval"] == 1]
temp_measure_check_int1

,HEAT_NO,int_tag,cnt_vcb_off,id_interval,heating_mwh_vcb,Temp_vcb_new,delta_T_calc,inc_count_0,inc_count,inc_count_max,id_str0,id_str,measure_temp_2
0,E238146,Normal_PwrOn_interval,1,1,5.101562,1539.0,78.0,1,1,1,measure_1,measure_2,1617.0
0,E238181,Normal_PwrOn_interval,1,1,3.898438,1540.0,91.0,1,1,1,measure_1,measure_2,1631.0
0,E238201,Normal_PwrOn_interval,1,1,5.000000,1520.0,79.0,1,1,1,measure_1,measure_2,1599.0
0,E238207,Normal_PwrOn_interval,3,1,4.503906,1529.0,31.0,1,1,3,measure_1,measure_2,1560.0
0,E238221,Normal_PwrOn_interval,2,1,3.898438,1588.0,0.0,1,1,2,measure_1,measure_2,1588.0
0,E239564,Normal_PwrOn_interval,2,1,4.398438,1554.0,13.0,1,1,2,measure_1,measure_2,1567.0
0,E239582,Normal_PwrOn_interval,1,1,0.000000,1584.0,0.0,0,0,0,measure_1,measure_2,1584.0
0,E239588,Normal_PwrOn_interval,1,1,7.003906,1516.0,90.0,1,1,1,measure_1,measure_2,1606.0
0,E239595,Normal_PwrOn_interval,1,1,3.699219,1524.0,68.0,1,1,1,measure_1,measure_2,1592.0
0,E239600,Normal_PwrOn_interval,1,1,5.597656,1524.0,82.0,1,1,1,measure_1,measure_2,1606.0


In [116]:
# 총 승온 전력량
temp_measure_check_n["total_input_mwh"] = temp_measure_check_n.groupby(["HEAT_NO"])["heating_mwh_vcb"].transform(lambda x : sum(x))

In [139]:
heating_tot_input_mwh = temp_measure_check_n[["HEAT_NO", "total_input_mwh"]].drop_duplicates()

check_mwh_merged = reduce(lambda x, y : pd.merge(x, y, on = ["HEAT_NO"]), [Total_heat_MWH_PX3, MWH_calculate, heating_tot_input_mwh])

#
check_mwh_merged["calculate_sum"] = check_mwh_merged[["charge_1_mwh", "charge_2_mwh", "charge_3_mwh", "total_input_mwh"]].sum(axis = 1)

#
check_mwh_diff = check_mwh_merged[["max", "calculate_sum"]]
check_mwh_diff["value_check"] = abs(check_mwh_diff["calculate_sum"] - check_mwh_diff["max"])
print("max : ", max(check_mwh_diff["value_check"]))
print("median : ", np.median(check_mwh_diff["value_check"]))
print("min : ", min(check_mwh_diff["value_check"]))

max :  0.09375
median :  0.00390625
min :  0.0


In [117]:
# 출강온도 
temp_measure_check_n["last_temp"] = temp_measure_check_n.groupby(["HEAT_NO"])["measure_temp_2"].transform(lambda x : max(x))

In [121]:
# 승온 후 온도 pivot
temp_measure_check_n_pivot = temp_measure_check_n.pivot_table(index = ["HEAT_NO", "inc_count_max"], 
                                                              columns = ["id_str"], 
                                                              values = ["measure_temp_2"]).reset_index()
temp_measure_check_n_pivot
temp_measure_check_n_pivot.columns = ["_".join(pair) for pair in temp_measure_check_n_pivot.columns]
temp_measure_check_n_pivot.rename(columns = {"HEAT_NO_" : "HEAT_NO",
                                             "inc_count_max_" : "inc_count_max"}, inplace = True)


# 측온항목값이 없는 경우 강제 생성
if "measure_temp_2_measure_3" not in temp_measure_check_n_pivot.columns :
    
    # 
    temp_measure_check_n_pivot["measure_temp_2_measure_3"] = np.nan
    
;

# 
if "measure_temp_2_measure_4" not in temp_measure_check_n_pivot.columns :
    
    # 
    temp_measure_check_n_pivot["measure_temp_2_measure_4"] = np.nan
    
;


#
if "measure_temp_2_measure_5" not in temp_measure_check_n_pivot.columns :
    
    # 
    temp_measure_check_n_pivot["measure_temp_2_measure_5"] = np.nan
    
;


#
if "measure_temp_2_measure_6" not in temp_measure_check_n_pivot.columns :
    
    # 
    temp_measure_check_n_pivot["measure_temp_2_measure_6"] = np.nan
    
;


#
if "measure_temp_2_measure_7" not in temp_measure_check_n_pivot.columns :
    
    # 
    temp_measure_check_n_pivot["measure_temp_2_measure_7"] = np.nan
    
;


# 
if "measure_temp_2_measure_8" not in temp_measure_check_n_pivot.columns :
    
    # 
    temp_measure_check_n_pivot["measure_temp_2_measure_8"] = np.nan
    
;
temp_measure_check_n_pivot

,HEAT_NO,inc_count_max,measure_temp_2_measure_2,measure_temp_2_measure_3,measure_temp_2_measure_4,measure_temp_2_measure_5,measure_temp_2_measure_6,measure_temp_2_measure_7,measure_temp_2_measure_8
0,E238146,1,1617.0,NaN,NaN,NaN,NaN,NaN,NaN
1,E238181,1,1631.0,NaN,NaN,NaN,NaN,NaN,NaN
2,E238201,1,1599.0,NaN,NaN,NaN,NaN,NaN,NaN
3,E238207,3,1560.0,1560.0,1560.0,NaN,NaN,NaN,NaN
4,E238221,2,1588.0,1597.0,NaN,NaN,NaN,NaN,NaN
5,E239564,2,1567.0,1587.0,NaN,NaN,NaN,NaN,NaN
6,E239582,0,1584.0,NaN,NaN,NaN,NaN,NaN,NaN
7,E239588,1,1606.0,NaN,NaN,NaN,NaN,NaN,NaN
8,E239595,1,1592.0,NaN,NaN,NaN,NaN,NaN,NaN
9,E239600,1,1606.0,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
# 첫 측온 온도 
temp_measure_check_int1.rename(columns = {"Temp_vcb_new" : "measure_temp_2_measure_1"}, inplace = True)

In [123]:
# 첫 측온온도와 병합
temp_measure_check_merge = pd.merge(temp_measure_check_int1[["HEAT_NO", "cnt_vcb_off", "measure_temp_2_measure_1"]], 
                                    temp_measure_check_n_pivot, on = "HEAT_NO")


In [124]:
temp_measure_check_merge

,HEAT_NO,cnt_vcb_off,measure_temp_2_measure_1,inc_count_max,measure_temp_2_measure_2,measure_temp_2_measure_3,measure_temp_2_measure_4,measure_temp_2_measure_5,measure_temp_2_measure_6,measure_temp_2_measure_7,measure_temp_2_measure_8
0,E238146,1,1539.0,1,1617.0,NaN,NaN,NaN,NaN,NaN,NaN
1,E238181,1,1540.0,1,1631.0,NaN,NaN,NaN,NaN,NaN,NaN
2,E238201,1,1520.0,1,1599.0,NaN,NaN,NaN,NaN,NaN,NaN
3,E238207,3,1529.0,3,1560.0,1560.0,1560.0,NaN,NaN,NaN,NaN
4,E238221,2,1588.0,2,1588.0,1597.0,NaN,NaN,NaN,NaN,NaN
5,E239564,2,1554.0,2,1567.0,1587.0,NaN,NaN,NaN,NaN,NaN
6,E239582,1,1584.0,0,1584.0,NaN,NaN,NaN,NaN,NaN,NaN
7,E239588,1,1516.0,1,1606.0,NaN,NaN,NaN,NaN,NaN,NaN
8,E239595,1,1524.0,1,1592.0,NaN,NaN,NaN,NaN,NaN,NaN
9,E239600,1,1524.0,1,1606.0,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
# 승온기 전력사용량(동일한 방법) 
mwh_measure_check_n_pivot = temp_measure_check_n.pivot_table(index = ["HEAT_NO", "inc_count_max", "total_input_mwh"], 
                                                             columns = ["id_str0"], 
                                                             values = ["heating_mwh_vcb"]).reset_index()

# 
mwh_measure_check_n_pivot.columns = ["_".join(pair) for pair in mwh_measure_check_n_pivot.columns]
mwh_measure_check_n_pivot.rename(columns = {"HEAT_NO_" : "HEAT_NO",
                                             "inc_count_max_" : "inc_count_max",
                                             "total_input_mwh_" : "total_input_mwh"}, inplace = True)


# 측온항목값이 없는 경우 강제 생성
if "heating_mwh_vcb_measure_3" not in mwh_measure_check_n_pivot.columns :
    
    # 
    mwh_measure_check_n_pivot["heating_mwh_vcb_measure_3"] = np.nan
    
;

# 
if "heating_mwh_vcb_measure_4" not in mwh_measure_check_n_pivot.columns :
    
    # 
    mwh_measure_check_n_pivot["heating_mwh_vcb_measure_4"] = np.nan
    
;


#
if "heating_mwh_vcb_measure_5" not in mwh_measure_check_n_pivot.columns :
    
    # 
    mwh_measure_check_n_pivot["heating_mwh_vcb_measure_5"] = np.nan
    
;


#
if "heating_mwh_vcb_measure_6" not in mwh_measure_check_n_pivot.columns :
    
    # 
    mwh_measure_check_n_pivot["heating_mwh_vcb_measure_6"] = np.nan
    
;


#
if "heating_mwh_vcb_measure_7" not in mwh_measure_check_n_pivot.columns :
    
    # 
    mwh_measure_check_n_pivot["heating_mwh_vcb_measure_7"] = np.nan
    
;


# 
if "heating_mwh_vcb_measure_8" not in mwh_measure_check_n_pivot.columns :
    
    # 
    mwh_measure_check_n_pivot["heating_mwh_vcb_measure_8"] = np.nan
    
;


''

In [126]:
# 필요 항목만 별도 지정

# 
temp_col_select = ["measure_temp_2_measure_1", "measure_temp_2_measure_2", 
                   "measure_temp_2_measure_3", "measure_temp_2_measure_4"]

# 
mwh_col_select = ["heating_mwh_vcb_measure_1", "heating_mwh_vcb_measure_2", 
                  "heating_mwh_vcb_measure_3", "heating_mwh_vcb_measure_4"]

In [127]:
# 데이터 병합
steelinfo_merged = reduce(lambda x, y : pd.merge(x, y, on = "HEAT_NO", how = "left"), 
                          [steelinfo_dt,
                           MWH_calculate[["HEAT_NO", "charge_1_mwh", "charge_2_mwh", "charge_3_mwh", "measure_last_melt_mwh"]],
                           temp_measure_check_merge[["HEAT_NO"] + temp_col_select],
                           mwh_measure_check_n_pivot[["HEAT_NO"] + mwh_col_select]])

# 
steelinfo_merged.rename(columns = {"charge_1_mwh" : "장입1차_용해전력(MWH)",
                                   "charge_2_mwh" : "장입2차_용해전력(MWH)",
                                   "charge_3_mwh" : "장입3차_용해전력(MWH)",
                                   "measure_last_melt_mwh" : "용락시점 총 투입 용해전력(MWH)",
                                   "total_input_mwh" : "승온기 총 전력사용량(MWH)", 
                                   "measure_temp_2_measure_1" : "1차측온온도",
                                   "measure_temp_2_measure_2" : "2차측온온도",
                                   "measure_temp_2_measure_3" : "3차측온온도",
                                   "measure_temp_2_measure_4" : "4차측온온도",
                                   "heating_mwh_vcb_measure_1" : "1차승온전력(MWH)",
                                   "heating_mwh_vcb_measure_2" : "2차승온전력(MWH)",
                                   "heating_mwh_vcb_measure_3" : "3차승온전력(MWH)",
                                   "heating_mwh_vcb_measure_4" : "4차승온전력(MWH)"}, inplace = True)

steelinfo_merged

,NO,생산일자,HEAT_NO,생산강종,용강특성,반,통전패턴,조업시종_개시,조업시종_종료,TAP_TO_TAP_계,TAP_TO_TAP_보수,TAP_TO_TAP_장입,TAP_TO_TAP_용해,TAP_TO_TAP_승온,TAP_TO_TAP_출강,정지,ONTAP,전력_KWh,전력_MW,산소_로벽,산소_로전,산소_수동,산소_계,LNG,장입횟수,장입량,출강량,생산량,회수율,출강온도,자동측온_온도,자동측온_산소,자동측온_C,자동측온_전력,자동측온_SLAG_T_Fe_용락,자동측온_SLAG_T_Fe_산말,산소개공,전극수명_1,전극수명_2,전극수명_3,EAF_PPM_산소1,EAF_PPM_탄소1,EAF_PPM_산소2,EAF_PPM_탄소2,EAF_PPM_산소3,EAF_PPM_탄소3,비상경동테스트_퀵백,비상경동테스트_후경,전기로투입부원료_생석회,전기로투입부원료_경소,전기로투입부원료_가탄괴,전기로투입부원료_가탄분,전기로투입부원료_알믹스,전기로투입부원료_폐연와,래들투입부원료_합금철_생석회,래들투입부원료_합금철_경소,래들투입부원료_합금철_가탄괴,래들투입부원료_합금철_가탄분,래들투입부원료_합금철_알믹스,래들투입부원료_합금철_형석,래들투입부원료_합금철_FeSi,래들투입부원료_합금철_SiMn,래들투입부원료_합금철_Al,전기로_래들합계_생석회,전기로_래들합계_경소,전기로_래들합계_가탄괴,전기로_래들합계_가탄분,전기로_래들합계_알믹스,전기로_래들합계_폐연와,전기로_래들합계_형석,전기로_래들합계_FeSi,전기로_래들합계_SiMn,전기로_래들합계_Al,장입1차_용해전력(MWH),장입2차_용해전력(MWH),장입3차_용해전력(MWH),용락시점 총 투입 용해전력(MWH),1차측온온도,2차측온온도,3차측온온도,4차측온온도,1차승온전력(MWH),2차승온전력(MWH),3차승온전력(MWH),4차승온전력(MWH)
0,1.0,2023.01.02,E235373,SS275A36(T<25),NaN,D,10.0,0.09,0.38,69.0,0.0,4.0,47.0,16.0,2.0,108.0,63.0,44100.0,42.0,503.0,2053.0,NaN,2556.0,1.0,3.0,102.5,81.3,83.10,81.1,1610.0,1610.0,880.0,3.0,44100.0,NaN,NaN,0.0,11968.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1811.0,821.0,617.0,997.0,723.0,NaN,NaN,208.0,NaN,NaN,NaN,NaN,151.0,220.0,NaN,1811.0,1029.0,617.0,997.0,723.0,NaN,NaN,151.0,220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2023.01.02,E235374,SS275A36(T<25),NaN,D,10.0,0.37,0.42,67.0,0.0,5.0,49.0,11.0,2.0,NaN,60.0,36700.0,36.7,586.0,1503.0,NaN,2089.0,1.0,3.0,100.2,86.1,82.80,82.6,1563.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,805.0,610.0,617.0,765.0,277.0,NaN,NaN,208.0,NaN,NaN,NaN,NaN,148.0,208.0,NaN,805.0,818.0,617.0,765.0,277.0,NaN,NaN,148.0,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,2023.01.02,E235375,SS275A36(T<25),NaN,D,10.0,0.42,0.46,55.0,0.0,6.0,36.0,11.0,2.0,NaN,47.0,36000.0,46.0,568.0,1643.0,NaN,2211.0,1.0,3.0,97.8,86.1,91.17,93.2,1583.0,1583.0,583.0,5.0,36000.0,NaN,NaN,0.0,11970.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,805.0,612.0,517.0,669.0,374.0,NaN,NaN,204.0,NaN,NaN,NaN,NaN,148.0,210.0,NaN,805.0,816.0,517.0,669.0,374.0,NaN,NaN,148.0,210.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2023.01.02,E235376,SS275A36(T<25),NaN,D,10.0,0.46,0.51,69.0,4.0,5.0,36.0,23.0,1.0,NaN,59.0,42800.0,43.5,616.0,1986.0,NaN,2602.0,2.0,4.0,112.1,94.7,97.75,87.2,1599.0,1583.0,583.0,5.0,31300.0,NaN,NaN,0.0,11971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,807.0,413.0,615.0,744.0,693.0,NaN,NaN,205.0,NaN,NaN,NaN,NaN,148.0,218.0,NaN,807.0,618.0,615.0,744.0,693.0,NaN,NaN,148.0,218.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,2023.01.02,E235377,SS275A36(T<25),NaN,D,10.0,0.50,0.56,71.0,12.0,4.0,36.0,19.0,NaN,NaN,55.0,36100.0,39.4,573.0,1538.0,NaN,2111.0,2.0,3.0,95.5,83.9,92.82,97.2,1544.0,1599.0,828.0,3.0,31000.0,NaN,NaN,0.0,11972.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,606.0,617.0,616.0,724.0,NaN,NaN,NaN,207.0,NaN,NaN,NaN,NaN,148.0,211.0,NaN,606.0,824.0,616.0,724.0,NaN,NaN,NaN,148.0,211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4641,456.0,2023.10.30,E240014,SHN355(T<25)V,NaN,C,10.0,0.40,0.45,62.0,3.0,5.0,39.0,13.0,2.0,NaN,52.0,34500.0,39.8,714.0,1689.0,NaN,2403.0,1.0,3.0,96.8,86.2,67.53,69.8,1574.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16623.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,401.0,208.0,507.0,685.0,536.0,NaN,402.0,413.0,NaN,NaN,NaN,NaN,100.0,533.0,NaN,803.0,621.0,507.0,685.0,536.0,NaN,NaN,100.0,533.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4642,457.0,2023.10.30,E240015,SM355A(T<25),NaN,C,10.0,0.44,0.48,52.0,0.0,5.0,35.0,10.0,2.0,NaN,45.0,34600.0,46.1,644.0,1484.0,NaN,2128.0,1.0,3.0,96.3,86.2,120.11,124.8,1606.0,1606.